# Claude Code Hooks System Documentation

**Type**: How-to Guide + Reference
**Last Updated**: 2025-09-21
**Version**: 1.0.0

## Overview

This documentation covers the Claude Code hooks system located in `.claude/hooks/`, which provides automated quality control, code formatting, and git protection for development workflows. The system automatically triggers after Claude responses, sub-agent completions, and session ends to maintain code quality standards.

## Prerequisites

### Required Knowledge
- Basic Python programming
- Git version control concepts
- Claude Code CLI usage
- Code quality tools (black, isort, ruff)

### System Requirements
- Python 3.11+
- Git installed and configured
- Claude Code CLI
- Python packages: black, isort, ruff, pydocstyle

### Dependencies
```bash
pip install black isort ruff pydocstyle
```

## Hook System Architecture

```mermaid
graph TB
    A[Claude Response] --> B[Hook Trigger]
    B --> C{Hook Type}
    C -->|Stop| D[post-response-quality-check]
    C -->|SubagentStop| D
    C -->|SessionEnd| E[session-end-quality-check]
    C -->|PostToolUse| F[post-file-edit-unicode-cleanup]
    
    D --> G[Quality Analysis]
    E --> G
    F --> H[Unicode Cleanup]
    
    G --> I{Issues Found?}
    I -->|Yes| J[quality-remediation-trigger]
    I -->|No| K[Complete]
    
    J --> L[Git Protection Commit]
    L --> M[Run Auto-fixes]
    M --> N[Compilation Check]
    N -->|Pass| O[Completion Commit]
    N -->|Fail| P[Rollback]
```

## How-to: Configure Hooks

### Step 1: Enable Hooks in Settings

Edit `.claude/settings.json` to configure which hooks should run:

In [ ]:
# Example hook configuration
hook_config = {
    "hooks": {
        "Stop": [
            {
                "command": "python",
                "args": [".claude/hooks/post-response-quality-check.py"],
                "description": "Run quality checks after Claude response"
            }
        ],
        "SubagentStop": [
            {
                "command": "python",
                "args": [".claude/hooks/post-response-quality-check.py"],
                "description": "Run quality checks after sub-agent completion"
            }
        ],
        "SessionEnd": [
            {
                "command": "python",
                "args": [".claude/hooks/session-end-quality-check.py"],
                "description": "Run comprehensive quality check at session end"
            }
        ],
        "PostToolUse": [
            {
                "command": "python",
                "args": [".claude/hooks/post-file-edit-unicode-cleanup.py"],
                "description": "Clean Unicode after file edits"
            }
        ]
    }
}

print("Hook configuration ready for .claude/settings.json")

### Step 2: Restart Claude Code CLI

After modifying settings.json, restart the CLI for hooks to take effect:

```bash
# Exit current session
exit

# Start new session
claude code
```

## How-to: Test Hooks Manually

### Testing Post-Response Quality Check

In [ ]:
import json
import subprocess

def test_post_response_hook():
    """Test the post-response quality check hook manually."""
    test_input = {
        "response_id": "test-123",
        "hook_type": "Stop"
    }
    
    cmd = ["python", ".claude/hooks/post-response-quality-check.py"]
    result = subprocess.run(
        cmd,
        input=json.dumps(test_input),
        text=True,
        capture_output=True
    )
    
    print("Hook output:")
    print(result.stdout)
    
    if result.returncode == 0:
        print("✅ Hook executed successfully")
    else:
        print(f"❌ Hook failed with code {result.returncode}")
        print(result.stderr)

# Run test
test_post_response_hook()

### Testing Unicode Cleanup Hook

In [ ]:
def test_unicode_cleanup_hook():
    """Test the Unicode cleanup hook after file edit."""
    test_input = {
        "tool_name": "Write",
        "tool_input": {
            "file_path": "test_file.py"
        }
    }
    
    cmd = ["python", ".claude/hooks/post-file-edit-unicode-cleanup.py"]
    result = subprocess.run(
        cmd,
        input=json.dumps(test_input),
        text=True,
        capture_output=True
    )
    
    print("Unicode cleanup output:")
    print(result.stdout)
    
    return result.returncode == 0

# Run test
if test_unicode_cleanup_hook():
    print("✅ Unicode cleanup hook working")
else:
    print("❌ Unicode cleanup hook failed")

## How-to: Monitor Hook Activity

### Check Quality Check Results

In [ ]:
from pathlib import Path
import glob
import datetime

def monitor_quality_checks():
    """Monitor recent quality check activity."""
    quality_dir = Path(".claude/quality-checks")
    
    if not quality_dir.exists():
        print("No quality checks directory found")
        return
    
    # Find recent quality files (last 24 hours)
    cutoff_time = datetime.datetime.now() - datetime.timedelta(days=1)
    
    patterns = [
        "post-response-quality-*.json",
        "session-end-quality-*.json",
        "triggers/remediation-trigger-*.json"
    ]
    
    recent_files = []
    for pattern in patterns:
        for file in quality_dir.glob(pattern):
            if file.stat().st_mtime > cutoff_time.timestamp():
                recent_files.append(file)
    
    if recent_files:
        print(f"Found {len(recent_files)} recent quality check files:")
        for file in sorted(recent_files, key=lambda x: x.stat().st_mtime, reverse=True):
            mtime = datetime.datetime.fromtimestamp(file.stat().st_mtime)
            print(f"  - {file.name} ({mtime.strftime('%Y-%m-%d %H:%M:%S')})")
    else:
        print("No recent quality check activity")

monitor_quality_checks()

### Check Hook Logs

In [ ]:
def view_hook_logs(lines=20):
    """View recent hook trigger logs."""
    log_file = Path(".claude/quality-checks/hooks-trigger.log")
    
    if not log_file.exists():
        print("No hook trigger log found")
        return
    
    with open(log_file, 'r', encoding='utf-8') as f:
        all_lines = f.readlines()
        recent_lines = all_lines[-lines:] if len(all_lines) > lines else all_lines
        
        print(f"Last {lines} lines from hook trigger log:")
        print("-" * 80)
        for line in recent_lines:
            print(line.rstrip())
        print("-" * 80)

view_hook_logs()

## How-to: Trigger Quality Remediation

### Manual Remediation Trigger

In [ ]:
def trigger_quality_remediation():
    """Manually trigger the quality remediation workflow."""
    import subprocess
    import json
    import os
    
    trigger_input = {
        "session_id": "manual-trigger",
        "cwd": os.getcwd(),
        "reason": "manual-quality-check"
    }
    
    cmd = ["python", ".claude/hooks/quality-remediation-trigger.py"]
    
    print("Triggering quality remediation workflow...")
    result = subprocess.run(
        cmd,
        input=json.dumps(trigger_input),
        text=True,
        capture_output=True,
        timeout=120
    )
    
    print("\nOutput:")
    print(result.stdout)
    
    if result.stderr:
        print("\nErrors:")
        print(result.stderr)
    
    return result.returncode == 0

# Trigger remediation
if trigger_quality_remediation():
    print("\n✅ Quality remediation completed successfully")
else:
    print("\n❌ Quality remediation failed")

## Reference: Hook Types and Triggers

### Hook Types

| Hook Type | Trigger Event | Primary Function |
|-----------|--------------|------------------|
| Stop | After Claude main response | Quality check on recent code changes |
| SubagentStop | After sub-agent completion | Quality check on sub-agent changes |
| SessionEnd | When session terminates | Comprehensive quality report |
| PostToolUse | After file modification tools | Unicode cleanup for Windows compatibility |

### File Modification Tools

Tools that trigger PostToolUse hooks:
- Write
- Edit
- MultiEdit
- NotebookEdit

## Reference: Quality Standards

### Code Quality Metrics

| Metric | Standard | Tool |
|--------|----------|------|
| Line Length | 120 characters | black |
| Import Sorting | black-compatible profile | isort |
| Code Formatting | PEP 8 compliant | black |
| Linting | Standard rules | ruff |
| Complexity (Non-UI) | < 10 | radon |
| Complexity (UI) | < 15 | radon |
| Docstrings | Google/NumPy style | pydocstyle |

### Git Protection Workflow

1. **Protection Commit**: Save current state before changes
2. **Auto-fixes**: Apply black, isort, ruff --fix
3. **Compilation Check**: Verify all Python files compile
4. **Decision Point**:
   - If compilation passes → Create completion commit
   - If compilation fails → Rollback to protection commit

## Reference: Hook Scripts

### Main Hook Scripts

#### post-response-quality-check.py
- **Purpose**: Quality check after Claude responses
- **Triggers**: Stop, SubagentStop events
- **Actions**: 
  - Scans files modified in last 5 minutes
  - Runs black formatter check
  - Triggers remediation if issues found

#### quality-remediation-trigger.py
- **Purpose**: Orchestrate automated quality fixes
- **Features**:
  - Git protection commits
  - Automated fixing (black, isort, ruff)
  - Compilation verification
  - Auto-rollback on failure

#### post-file-edit-unicode-cleanup.py
- **Purpose**: Remove Unicode/emoji after file edits
- **Triggers**: PostToolUse events
- **Supported files**: .py, .md, .txt, .yml, .json, .js, .ts

#### session-end-quality-check.py
- **Purpose**: Comprehensive quality report at session end
- **Features**:
  - Full codebase scan
  - Detailed metrics
  - Summary report generation

## Reference: Tool Scripts

### tools/python_quality_manager.py
- **Purpose**: Python code quality management
- **Features**:
  - pyproject.toml configuration discovery
  - Compilation checking
  - Complexity analysis
  - Docstring validation

### tools/git_protection_manager.py
- **Purpose**: Git-based change protection
- **Features**:
  - Protection commits
  - Completion commits
  - Rollback functionality
  - Change tracking

### tools/unicode_manager.py
- **Purpose**: Unicode character cleanup
- **Features**:
  - Emoji removal
  - Unicode replacement
  - Windows compatibility
  - Safe file processing

### tools/pattern_scanner.py
- **Purpose**: Code pattern detection
- **Features**:
  - AST-based analysis
  - Pattern matching
  - Issue detection

### tools/pattern_fixer.py
- **Purpose**: Automated pattern fixes
- **Features**:
  - Safe AST transformations
  - Pattern correction
  - Code preservation

## Troubleshooting

### Common Issues and Solutions

#### Hooks Not Triggering
**Problem**: Hooks configured but not executing automatically
**Solution**: 
1. Verify settings.json syntax is correct
2. Restart Claude Code CLI completely
3. Check hook script permissions
4. Test hooks manually to verify functionality

#### Quality Files Not Created
**Problem**: No quality check JSON files appearing
**Solution**:
1. Ensure `.claude/quality-checks/` directory exists
2. Check write permissions
3. Look for errors in hooks-trigger.log
4. Run manual test to verify scripts work

#### Compilation Failures After Auto-fix
**Problem**: Code doesn't compile after automated fixes
**Solution**:
1. System automatically rollbacks to protection commit
2. Check remediation logs for specific errors
3. Manually review and fix compilation issues
4. Re-run quality check after manual fixes

#### Git Protection Commits Missing
**Problem**: No protection/completion commits in git log
**Solution**:
1. Verify git is properly configured
2. Check if repository is in clean state
3. Ensure git_protection_manager.py is accessible
4. Review git permissions and credentials

## Examples

### Example: Full Quality Workflow

In [ ]:
def demonstrate_full_workflow():
    """Demonstrate the complete quality workflow."""
    import tempfile
    import os
    from pathlib import Path
    
    # Create a test Python file with quality issues
    test_file = Path("test_quality_demo.py")
    
    # Write code with intentional quality issues
    bad_code = '''
import os,sys,json
from pathlib import   Path


def   badly_formatted_function(  x,y,  z   ):
    """Missing proper docstring"""
    result=x+y*z
    if result>100:
        print( "Result is large"   )
    return   result


class  PoorlyFormattedClass:
    def __init__(self,name,value):
        self.name=name
        self.value=value
    def display(   self  ):
        print(f"{self.name}: {self.value}")
'''
    
    # Write the bad code to file
    with open(test_file, 'w') as f:
        f.write(bad_code)
    
    print(f"Created test file with quality issues: {test_file}")
    print("\nOriginal code:")
    print("-" * 40)
    print(bad_code)
    print("-" * 40)
    
    # Now trigger quality check
    print("\nTriggering quality check...")
    import subprocess
    result = subprocess.run(
        ["python", "-m", "black", "--check", "--diff", str(test_file)],
        capture_output=True,
        text=True
    )
    
    if result.returncode != 0:
        print("\nBlack found formatting issues:")
        print(result.stdout[:500])  # Show first 500 chars of diff
        
        # Apply black formatting
        print("\nApplying black formatter...")
        subprocess.run(["python", "-m", "black", str(test_file)])
        
        # Show fixed code
        with open(test_file, 'r') as f:
            fixed_code = f.read()
        
        print("\nFixed code:")
        print("-" * 40)
        print(fixed_code)
        print("-" * 40)
    
    # Clean up
    if test_file.exists():
        test_file.unlink()
        print(f"\nCleaned up test file: {test_file}")

# Run demonstration
demonstrate_full_workflow()

### Example: Custom Hook Creation

In [ ]:
def create_custom_hook_template():
    """Generate a template for creating custom hooks."""
    
    template = '''#!/usr/bin/env python3
"""
Custom Hook Template
====================
Description: [Your hook description here]
Hook Type: [Stop|SubagentStop|SessionEnd|PostToolUse]
Triggers: [When this hook runs]
"""

import json
import sys
from datetime import datetime
from pathlib import Path

def log_message(message, level="INFO"):
    """Create timestamped log message."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return f"[{timestamp}] [{level}] {message}"

def main():
    """Main hook execution."""
    try:
        # Read hook input from stdin
        input_data = {}
        if not sys.stdin.isatty():
            try:
                input_text = sys.stdin.read().strip()
                if input_text:
                    input_data = json.loads(input_text)
            except json.JSONDecodeError as e:
                print(log_message(f"Invalid JSON input: {e}", "ERROR"))
                return 1
        
        # Extract hook context
        hook_type = input_data.get("hook_type", "Unknown")
        response_id = input_data.get("response_id", "standalone")
        
        print(log_message(f"Custom hook triggered - Type: {hook_type}, ID: {response_id[:8]}"))
        
        # YOUR CUSTOM LOGIC HERE
        # Example: Perform some action based on hook type
        if hook_type == "Stop":
            # Handle Stop event
            print(log_message("Processing Stop event"))
            # Add your logic here
        
        elif hook_type == "SubagentStop":
            # Handle SubagentStop event
            print(log_message("Processing SubagentStop event"))
            # Add your logic here
        
        elif hook_type == "SessionEnd":
            # Handle SessionEnd event
            print(log_message("Processing SessionEnd event"))
            # Add your logic here
        
        else:
            print(log_message(f"Unknown hook type: {hook_type}", "WARN"))
        
        # Return success
        print(log_message("Custom hook completed successfully"))
        return 0
        
    except Exception as e:
        print(log_message(f"Hook execution failed: {e}", "ERROR"))
        return 1

if __name__ == "__main__":
    sys.exit(main())
'''
    
    print("Custom Hook Template:")
    print("=" * 60)
    print(template)
    print("=" * 60)
    print("\nTo use this template:")
    print("1. Save to .claude/hooks/your-custom-hook.py")
    print("2. Add your custom logic in the main() function")
    print("3. Configure in .claude/settings.json")
    print("4. Restart Claude Code CLI")

create_custom_hook_template()

## Related Resources

### Internal Documentation
- `.claude/hooks/HANDOVER_DOCUMENTATION.md` - Session handover details
- `.claude/hooks/CONTEXT_PRESERVATION.md` - Context preservation guide
- `.claude/hooks/SESSION_HANDOVER_SUMMARY.md` - Summary documentation

### Configuration Files
- `.claude/settings.json` - Main settings with hook configuration
- `.claude/settings.local.json` - Local overrides
- `pyproject.toml` - Python project configuration

### Command Documentation
- `.claude/commands/quality/` - Quality command protocols
- `.claude/commands/code/` - Code command protocols
- `.claude/commands/protocol/` - Protocol enforcement

### External Tools
- [Black Documentation](https://black.readthedocs.io/)
- [isort Documentation](https://pycqa.github.io/isort/)
- [Ruff Documentation](https://docs.astral.sh/ruff/)
- [pydocstyle Documentation](http://www.pydocstyle.org/)

## Summary

The Claude Code hooks system provides comprehensive automated quality control for development workflows. Key capabilities include:

### Core Features
- **Automatic Triggering**: Hooks run after Claude responses, sub-agent completions, and session ends
- **Quality Enforcement**: Automated formatting, linting, and complexity checks
- **Git Protection**: Safe automation with protection commits and rollback capability
- **Unicode Cleanup**: Windows compatibility through automatic Unicode removal
- **Compilation Verification**: Ensures code remains functional after auto-fixes

### Best Practices
1. **Always restart CLI** after modifying hook configuration
2. **Monitor quality files** to track hook activity
3. **Test hooks manually** before relying on automation
4. **Review git commits** created by automation workflow
5. **Check compilation status** after automated fixes

### Safety Guarantees
- **No broken code**: Compilation checks prevent non-functional code
- **Full traceability**: Every change tracked with descriptive commits
- **Automatic recovery**: Rollback on any failure
- **Smart thresholds**: Different standards for UI vs non-UI code

The hooks system ensures code quality is maintained automatically throughout the development process, reducing manual overhead while maintaining high standards.